In [1]:
import getpass
import os
import json
import pandas as pd
import functools
import operator
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool 
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from typing import Sequence, TypedDict, Annotated

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

# 設置環境變量
_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("OPENAI_ORGANIZATION")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

# 读取CSV文件
file_path = "ptt_stock_posts_num1.csv"
df = pd.read_csv(file_path)

# 预览数据
print(df.head())

# 定義工具
tavily_tool = TavilySearchResults(max_results=5)
python_repl_tool = PythonREPLTool() 

# 定義代理
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    # 獲取原始內容
    original_content = state.get("original_content", "")
    original_pushes = state.get("original_pushes", "")
    # 獲取之前的消息歷史
    previous_messages = state["messages"]
    
    # 根據 agent 的角色構建新的輸入消息
    if name == "Analysis_1":         
        new_message = HumanMessage(content=f"This is a content: {original_content} and its pushes. Please analyze the sentiment of the following pushes: {original_pushes}")
    else:
        previous_analysis = previous_messages[-1].content if previous_messages else ""
        new_message = HumanMessage(content=f"Given the following original content,pushes and previous analysis, provide a comprehensive financial sentiment analysis with every pushes:\n\nOriginal content: {original_content}\n\nOriginal pushes:{original_pushes}\n\nPrevious analysis: {previous_analysis}")
    
    # 將新消息添加到狀態中
    updated_state = {
        "messages": previous_messages + [new_message],
        "original_content": original_content,
        "original_pushes": original_pushes
    }
    
    # 調用 agent
    result = agent.invoke(updated_state)
    
    # 返回更新後的狀態
    return {
        "messages": updated_state["messages"] + [HumanMessage(content=result["output"], name=name)],
        "original_content": original_content,
        "original_pushes": original_pushes
    }
    
    

members = ["Analysis_1", "Analysis_final"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [{"enum": options}],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4o")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    next: str
    original_content: str

research_agent = create_agent(llm, [tavily_tool], "You are a investor. Analyze the sentiment of the given financial content.")
research_node = functools.partial(agent_node, agent=research_agent, name="Analysis_1")

# code_agent = create_agent(llm,[python_repl_tool], "You may generate charts using matplotlib.")
# code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

finance_sentiment_agent = create_agent(llm, [tavily_tool], "You are a financial sentiment analysis expert. Analyze the sentiment of the given financial content.")
finance_sentiment_node = functools.partial(agent_node, agent=finance_sentiment_agent, name="Analysis_final")

workflow = StateGraph(AgentState)
workflow.add_node("Analysis_1", research_node) 
workflow.add_node("Analysis_final", finance_sentiment_node)
# workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

for member in members:
    workflow.add_edge(member, "supervisor")
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

# 流式處理每篇文章
for index, row in df.iterrows():
    article_pushes = row['Pushes']
    pushes_article_content = row['Content']
    
    # 將推文 JSON 字符串解析為字典
    pushes_dict = json.loads(article_pushes)
    
    # 提取推文內容
    pushes_list = [push["Content"] for push in pushes_dict.values()]
    
    # 分批處理推文並收集分析結果
    all_analysis_results = []
    for i in range(0, len(pushes_list), 50):
        batch_pushes = pushes_list[i:i + 50]
        batch_pushes_text = ' '.join(batch_pushes)
    
        input_message_content = HumanMessage(content=f"This is a content: {pushes_article_content} and its pushes. Please analyze the sentiment of the following pushes: {batch_pushes_text}")
    
        messages_dict = {
            "messages": [input_message_content],
            "original_content": pushes_article_content,
            "original_pushes": batch_pushes_text
        }
    
        for s in graph.stream(messages_dict):
            if "__end__" not in s:
                # 提取 AddableUpdatesDict 對象的輸出並轉換為字符串
                analysis_result = s["messages"][-1].content if "messages" in s else ""
                all_analysis_results.append(analysis_result)
                print(s)
                print("----")

    # 合併分析結果
    combined_analysis = " ".join(all_analysis_results)

    # 最終分析輸入
    #final_input_message_content = HumanMessage(content=f"Given the following original content, pushes, and previous analysis results, provide a comprehensive financial sentiment analysis:\n\nOriginal content: {pushes_article_content}\n\nOriginal pushes: {batch_pushes_text}\n\nPrevious analysis: {combined_analysis}")

    #final_messages_dict = {
    #    "messages": [final_input_message_content],
    #    "original_content": pushes_article_content,
    #    "original_pushes": batch_pushes_text
    #}

# 結合結果進行綜合分析
#    for s in graph.stream(final_messages_dict):
#        if "__end__" not in s:
 #           print(s)
#            print("----")


print('done')


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Please provide your OPENAI_API_KEY ········
Please provide your OPENAI_ORGANIZATION ········
Please provide your LANGCHAIN_API_KEY ········
Please provide your TAVILY_API_KEY ········


   Unnamed: 0                 Title           Author  \
0           0  [閒聊] 2024/07/12 盤中閒聊  vendan5566 (阿文)   

                       Date  \
0  Fri Jul 12 08:30:00 2024   

                                             Content  \
0  \n=============113/07/12台股資訊重點整理，供股民做投資參考=====...   

                                                Link  \
0  https://www.ptt.cc/bbs/Stock/M.1720744202.A.10...   

                                              Pushes  
0  {"0":{"Tag":"PUSH","Userid":"ZhanBro","Content...  
{'supervisor': {'next': 'Analysis_1'}}
----
{'Analysis_1': {'messages': [HumanMessage(content='This is a content: \n=============113/07/12台股資訊重點整理，供股民做投資參考============\n台股  24390.03▲382.95(+1.60%)成交量5569.31億\n台指0724367   ▲365   (+1.52%)成交量 92,984口\n盤後0724049   ▼322   (-1.32%)成交量 99,879口\n\n台指期未平倉口數自營商 -7,055(    -88)投信+22,681(  +714)外資-35,249( -1,621)\n台指選未平倉口數自營商 -7,659( +9,129)投信     +3(     0)外資 -9,832( +3,371)\n\n----------------------------------------\n道瓊工業39753.75▲ 32.39(0.08%